StackGAN implementation 

In [3]:
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import pickle
import random
import time

import numpy as np
import pandas as pd
import tensorflow as tf

assert tf.__version__.startswith('2')

import PIL
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, ReLU, Activation
from tensorflow.keras.layers import UpSampling2D, Conv2D, Concatenate, Dense, concatenate
from tensorflow.keras.layers import Flatten, Lambda, Reshape, ZeroPadding2D, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Next is implemetation of GAN network:
-> the imput text is then sent into the embedding layer (Word2vec Embedding -> here we do conditional augumentation)

*Conditional Augumentation*
the data is embedded with the computer language here (a nth-dimention array)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def conditioning_augmentation(x):
    mean = x[:, :128]
    log_sigma = x[:, :128]
    std = tf.math.exp(log_sigma)
    epsilon = K.random_normal(shape=(mean.shape[1],))
    c = mean + std * epsilon
    return c

def build_ca_network():
    input_layer1 = Input(shape=(1024,))
    mls = Dense(256)(input_layer1)
    mls = LeakyReLU(alpha=0.2)(mls)
    ca = Lambda(conditioning_augmentation)(mls)
    return Model(inputs=[input_layer1], outputs=[ca])


_After the conditioning augumentation the array of values is returned -> Stage 1_

*At stage 1, it takes text embedding -> generates the image*

In [34]:
# upsampling (also we can use conv2Dtranspose) -> used to increase the height and width of the inputed feature

def UpSamplingBlock(x, n_kernel):
    # n_kernels -> filters used
    # x -> the image that has to be upsampled
    x = UpSampling2D(size=(2, 2))(x)  # here we double the size of the image
    x = Conv2D(n_kernel, kernel_size=(3, 3), padding='same', strides=1, use_bias=False,
               kernel_initializer='he_uniform')(x)
    x = BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x) 
    x = ReLU()(x) 
    return x  # upsampled image

# batchnormalization -> used to prevent the collapse between the generated image of different text inputs
	
def stage1_generater():
    input_layer1 = Input(shape=(1024,))  # embedding vector
    ca = Dense(256)(input_layer1)  # with 256 neurons compressing the text embeddings

    ca = LeakyReLU(alpha=0.2)(ca)
    c = Lambda(conditioning_augmentation)(ca)
    input_layer2 = Input(shape=(100,))  # random number for shape
    # input_layer2 -> it is the random noise 
    # c -> embedding vector
    concat = Concatenate(axis=1)([c, input_layer2])  # the nth dimensional array

    x = Dense(16384, use_bias=False)(concat) 
    # 16384 = reshape (4,4,1024) + 4 * 4 * 1024
    x = ReLU()(x)
    x = Reshape((4, 4, 1024), input_shape=(16384,))(x)

    # the image resolution can be changed after the process that takes place in order to make the model
    # understand about the low resolution features to produce a high resolution image
    x = UpSamplingBlock(x, 512)  # doing upsampling from 4x4 image to 64x64 sized image
    x = UpSamplingBlock(x, 256)
    x = UpSamplingBlock(x, 128)
    x = UpSamplingBlock(x, 64)  # upsampled our image to 64*64*3, low resolution image

    x = Conv2D(3, kernel_size=3, padding='same', strides=1, use_bias=False,
               kernel_initializer='he_uniform')(x)
    x = Activation('tanh')(x)

    stage1_gen = Model(inputs=[input_layer1, input_layer2], outputs=[x, ca]) 
    return stage1_gen


In [35]:
gen = stage1_generater()
gen.summary()

D:\Languages\Python\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


D:\Languages\Python\Lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 1024)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 256)               │         262,400 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu (LeakyReLU)       │ (None, 256)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda (Lambda)               │ (None, 128)               │               0 │ leaky_re_lu[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_1 (InputLayer)    │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 228)               │               0 │ lambda[0][0],              │
│                               │                           │                 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 16384)             │       3,735,552 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu (ReLU)                  │ (None, 16384)             │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape (Reshape)             │ (None, 4, 4, 1024)        │               0 │ re_lu[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d (UpSampling2D)  │ (None, 8, 8, 1024)        │               0 │ reshape[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 8, 8, 512)         │       4,718,592 │ up_sampling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 8, 8, 512)         │           2,048 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_1 (ReLU)                │ (None, 8, 8, 512)         │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_1               │ (None, 16, 16, 512)       │               0 │ re_lu_1[0][0]              │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 16, 16, 256)       │       1,179,648 │ up_sampling2d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 16, 16, 256)       │           1,02

 Total params: 10,270,400 (39.18 MB)

 Trainable params: 10,268,480 (39.17 MB)

 Non-trainable params: 1,920 (7.50 KB)